# Import Modules

In [ ]:
%matplotlib inline

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers

from PIL import Image

tf.get_logger().setLevel('INFO')

data_dir = './data/data'

# Download Dataset

In [ ]:
# This cell is so someone can more easily acquire the dataset used
# Not necessary if done manually

import gdown
url = "https://drive.google.com/uc?id=117GBW1IkBOiKTqBnTJxE8E_Bh3J6Ha3g"
output = "data.7z"
gdown.download(url, output, quiet=False)

# Unpack Dataset

In [ ]:
from pyunpack import Archive
import os 
directory = './data'

try:
    os.mkdir(directory)
except Exception:
    pass

Archive('data.7z').extractall('./data/')

# Create Dataset Objects

In [ ]:
# Generates a tf.data.Dataset from image files in a directory.
# subsets are keywords, see api
# 80% of the images for training, and 20% for validation
# https://keras.io/api/preprocessing/image/

img_height = 160
img_width = 120
img_channels = 1

train_ds = tf.keras.preprocessing.image_dataset_from_directory(data_dir,
                                                    subset="training",
                                                    validation_split=0.21,
                                                    seed=np.random.randint(255),
                                                    image_size=(img_height,img_width),
                                                    color_mode='grayscale', 
                                                    batch_size=16) 

val_ds = tf.keras.preprocessing.image_dataset_from_directory(data_dir,
                                                    subset="validation",
                                                    validation_split=0.21,
                                                    seed=np.random.randint(255),
                                                    image_size=(img_height,img_width),
                                                    color_mode='grayscale',
                                                    batch_size=16)



# print class names
class_names = train_ds.class_names
print(class_names)

# give examples of images from dataset
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(6):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")
    
# Speed up by prefectching and caching during model creation
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Create Model

In [ ]:
num_classes = 2

model = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, img_channels)),
  layers.Dense(num_classes * 32, activation='relu'),
  layers.Conv2D(num_classes * 16, 3, strides=2, activation='relu'),
  layers.Dense(num_classes * 16, activation='relu'),
  layers.Dropout(0.5),
  layers.MaxPooling2D(),
  layers.Conv2D(num_classes * 8, 3, strides=2),
  layers.Dense(num_classes * 8, activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(num_classes * 4, activation='relu'),
  layers.Dense(num_classes, activation='relu')
])

learning_rate=0.001
opt = keras.optimizers.Adam(learning_rate=learning_rate)

model.compile(
  optimizer=opt,
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [ ]:
epochs = 20
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
  verbose=1
)
model.summary()

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(30, 30))
plt.subplot(2, 1, 1)
plt.grid(b=True, which='both', axis='both')
plt.xlabel('Epochs', fontsize='xx-large')
plt.ylabel('Accuracy(%)', fontsize='xx-large')
plt.plot(epochs_range, acc, label='Training Accuracy', linewidth=10)
plt.plot(epochs_range, val_acc, label='Validation Accuracy', linewidth=10)
plt.legend(loc='lower right', fontsize='xx-large')

plt.title('Training and Validation Accuracy', fontsize='xx-large')
plt.subplot(2, 1, 2)
plt.plot(epochs_range, loss, label='Training Loss', linewidth=10)
plt.grid(b=True, which='both', axis='both')
plt.xlabel('Epochs', fontsize='xx-large')
plt.ylabel('Accuracy(%)', fontsize='xx-large')

plt.plot(epochs_range, val_loss, label='Validation Loss', linewidth=10)
plt.legend(loc='upper right', fontsize='xx-large')
plt.title('Training and Validation Loss',fontsize='xx-large')
plt.show()


# Load model in lieu of training

In [ ]:
from tensorflow import keras

model = keras.models.load_model('./1/')

img_height = 160
img_width = 120
img_channels = 1

class_names=['fire_smoke', 'no-fire']

In [ ]:
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing import image
filename = './no-fire-test.jpg'

np_image = image.load_img(filename, color_mode='grayscale', target_size= (img_height, img_width, img_channels), interpolation='lanczos')
input_arr = image.img_to_array(np_image)
input_arr = np.array([input_arr])  # Convert single image to a batch.

plt.title('Image for Prediction',fontsize='xx-large')
plt.tick_params(
    axis='both',       # changes apply to the x-axis and y-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    left=False,        # ticks along the left edge are off
    right=False,       # ticks along the right edge are off
    labelleft=False,   # labels along the left edge are off
    labelbottom=False) # labels along the bottom edge are off

print(model.predict(input_arr))
img = mpimg.imread(filename)
imgplot = plt.imshow(img)
class_names[np.argmax(model.predict(input_arr))]

In [ ]:
filename = './fire.jpg'

np_image = image.load_img(filename, color_mode='grayscale', target_size= (img_height, img_width, img_channels), interpolation='lanczos')
input_arr = image.img_to_array(np_image)
input_arr = np.array([input_arr])  # Convert single image to a batch.

plt.title('Image for Prediction',fontsize='xx-large')
plt.tick_params(
    axis='both',       # changes apply to the x-axis and y-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    left=False,        # ticks along the left edge are off
    right=False,       # ticks along the right edge are off
    labelleft=False,   # labels along the left edge are off
    labelbottom=False) # labels along the bottom edge are off

print(model.predict(input_arr))
img = mpimg.imread(filename)
imgplot = plt.imshow(img)
class_names[np.argmax(model.predict(input_arr))]
